In [1]:
import sys
import getpass
import os
import pandas as pd
import fiona
import geopandas as gpd
from arcgis import gis
username = getpass.getuser()

user = getpass.getuser()
sys.dont_write_bytecode = True

# for DataViz team members
sys.path.insert(0, '/Users/{}/Box/DataViz Projects/Utility Code'.format(user))

from utils_io import *

In [2]:
client = gis.GIS(url='https://arcgis.ad.mtc.ca.gov/portal/home/',username=username,verify_cert=False)

Enter password: ········


In [3]:
transit_dir = os.path.join('/Users',
                        username,
                        'Box',
                        'DataViz Projects',
                        'Spatial Analysis and Mapping',
                        'Active Transportation Plan',
                        'Data',
                        'regional_transit')

In [65]:
active_trans_dir = os.path.join('/Users',
                        username,
                        'Box',
                        'DataViz Projects',
                        'Spatial Analysis and Mapping',
                        'Active Transportation Plan',
                        'Data',
                        'Active Transportation.gdb')

## Read most recent transit data from

In [51]:
transit_rts_precovid = pull_geodata_from_argis(arcgis_data_id='2a519083d0a44a33940e469e427c8457',
                                               client=client)

In [66]:
transit_rts_current = gpd.read_file(active_trans_dir,
                            driver='FileGDB',
                            layer='regional_transit_routes_oct_2021')

In [67]:
#Rename current columns to match precovid columns
rename_dict = {'route_short_name':'route_s_nm',
               'route_long_name':'route_l_nm',
               'route_type_text':'route_type'}
transit_rts_current.rename(columns=rename_dict,inplace=True)

In [68]:
transit_rts_current.rename_geometry(col='geom',inplace=True)

In [69]:
#Add col to pre-covid routes to indicate vintage
transit_rts_precovid['vintage'] = 'January 2020'

In [70]:
#Add col to current routes to indicate vintage
transit_rts_current['vintage'] = 'October 2021'

In [71]:
#Drop Napa Valley Vine Transit pre-covid route E
transit_rts_precovid.drop(transit_rts_precovid[transit_rts_precovid['route_id'] == 'VN:E'].index,
                          inplace=True)

In [72]:
#Subset new Napa Valley Vine Transit Routes
out_cols = ['route_id',
            'agency_id',
            'route_s_nm',
            'route_l_nm',
            'route_type',
            'vintage',
            'geom']
trans_rts_current_vn = transit_rts_current[out_cols][transit_rts_current['route_id']
                                                     .isin(['VN:E','VN:S','VN:N','VN:W'])]

In [73]:
transit_rts_precovid.head(5)

,SHAPE__Length,agency_id,agency_nm,objectid,route_id,route_l_nm,route_s_nm,route_type,geom,vintage
0,0.185912,SR,Santa Rosa CityBus,1,SR:4,Rincon Valley,4,Bus,"MULTILINESTRING ((-122.71373 38.43869, -122.71...",January 2020
1,0.185912,SR,Santa Rosa CityBus,2,SR:4,Rincon Valley,4,Bus,"MULTILINESTRING ((-122.71373 38.43869, -122.71...",January 2020
2,0.074137,SR,Santa Rosa CityBus,3,SR:5,Petaluma Hill Rd,5,Bus,"MULTILINESTRING ((-122.71371 38.43869, -122.71...",January 2020
3,0.059102,SR,Santa Rosa CityBus,4,SR:1,Mendocino Ave,1,Bus,"MULTILINESTRING ((-122.73259 38.45713, -122.73...",January 2020
4,0.225273,SC,VTA,5,SC:26,West Valley College - Eastridge,26,Bus,"MULTILINESTRING ((-121.81099 37.32716, -121.81...",January 2020


In [74]:
trans_rts_current_vn.head(5)

,route_id,agency_id,route_s_nm,route_l_nm,route_type,route_type,vintage,geom
2970,VN:N,VN,N,Trancas | North Napa,3,Bus,October 2021,"MULTILINESTRING ((-122.28026 38.29754, -122.28..."
2971,VN:N,VN,N,Trancas | North Napa,3,Bus,October 2021,"MULTILINESTRING ((-122.31050 38.32224, -122.31..."
2972,VN:S,VN,S,Imola | South Napa,3,Bus,October 2021,"MULTILINESTRING ((-122.27984 38.29752, -122.27..."
2973,VN:S,VN,S,Imola | South Napa,3,Bus,October 2021,"MULTILINESTRING ((-122.30419 38.28765, -122.30..."
2975,VN:W,VN,W,West,3,Bus,October 2021,"MULTILINESTRING ((-122.28000 38.29751, -122.27..."


In [75]:
transit_rts_precovid[out_cols]

,route_id,agency_id,route_s_nm,route_l_nm,route_type,vintage,geom
0,SR:4,SR,4,Rincon Valley,Bus,January 2020,"MULTILINESTRING ((-122.71373 38.43869, -122.71..."
1,SR:4,SR,4,Rincon Valley,Bus,January 2020,"MULTILINESTRING ((-122.71373 38.43869, -122.71..."
2,SR:5,SR,5,Petaluma Hill Rd,Bus,January 2020,"MULTILINESTRING ((-122.71371 38.43869, -122.71..."
3,SR:1,SR,1,Mendocino Ave,Bus,January 2020,"MULTILINESTRING ((-122.73259 38.45713, -122.73..."
4,SC:26,SC,26,West Valley College - Eastridge,Bus,January 2020,"MULTILINESTRING ((-121.81099 37.32716, -121.81..."
...,...,...,...,...,...,...,...
5926,WH:70X,WH,70X,Pleasant Hill,Bus,January 2020,"MULTILINESTRING ((-121.89941 37.70265, -121.89..."
5927,WH:70X,WH,70X,Pleasant Hill,Bus,January 2020,"MULTILINESTRING ((-122.05602 37.92812, -122.05..."
5928,WH:70X,WH,70X,Pleasant Hill,Bus,January 2020,"MULTILINESTRING ((-121.89997 37.70308, -121.89..."
5929,WH:70X,WH,70X,Pleasant Hill,Bus,January 2020,"MULTILINESTRING ((-121.89941 37.70265, -121.89..."


In [76]:
transit_rts_precov_nv = pd.concat([transit_rts_precovid[out_cols],trans_rts_current_vn],axis=0,
                                  ignore_index=True,
                                  sort=False)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

## Read transit routes classification xlsx

In [ ]:
os.listdir(transit_dir)

In [ ]:
transit_class = pd.read_excel(os.path.join(transit_dir,'BA_Agency_Category_0821.xlsx'),
                              sheet_name=2)

In [ ]:
transit_class.info()

In [ ]:
transit_rts_current[transit_rts_current['agency_id'] == 'CE']

In [ ]:
#Fix BART route ids to match shape ids
ba_dict = {'BA:BG-N': 'BA:Beige',
 'BA:BG-S': 'BA:Beige',
 'BA:BL-N': 'BA:Blue-Wkd/Sat',
 'BA:BL-S': 'BA:Blue-Wkd/Sat',
 'BA:GN-N': 'BA:Green',
 'BA:GN-S': 'BA:Green',
 'BA:OR-N': 'BA:Orange',
 'BA:OR-S': 'BA:Orange',
 'BA:PR-N': 'BA:Purple',
 'BA:PR-S': 'BA:Purple',
 'BA:RD-N': 'BA:Red',
 'BA:RD-S': 'BA:Red',
 'BA:YL-N': 'BA:Yellow',
 'BA:YL-S': 'BA:Yellow'}

transit_class['route_id'].replace(ba_dict,inplace=True)

In [ ]:
transit_rts_class = pd.merge(transit_rts_precovid[['route_id','geom']],
                             transit_class,
                             how='outer',
                             on='route_id',
                             indicator=True)

## Write non-match files to csv

In [ ]:
transit_rts_class[transit_rts_class['_merge'] == 'right_only']['agency_id'].unique()

In [ ]:
transit_rts_class[transit_rts_class['_merge'] == 'both']

In [ ]:
class_out_cols = ['agency_id',
                  'agency',
                  'route_short_name',
                  'route_long_name',
                  'category',
                  'sub_category',
                  'avg_peak_freq (min)',
                  'avg_offpeak_freq (min)',
                  'notes/comments']
class_export_df = transit_rts_class[class_out_cols][transit_rts_class['_merge'] == 'right_only'].copy()

In [ ]:
agency_ids = ['CE', 'CT', 'CC', 'SM', 'SF', 'VN', 'SC']
shape_out_cols = ['agency_id',
                  'agency_nm',
                  'route_id',
                  'route_l_nm',
                  'route_s_nm',
                  'route_type']
shape_export_df = transit_rts[shape_out_cols][transit_rts['agency_id'].isin(agency_ids)].copy()

In [ ]:
with pd.ExcelWriter(os.path.join(transit_dir,'nonmatching_route_ids.xlsx')) as writer:
    class_export_df.to_excel(writer, sheet_name='Unmatched_Route_Ids',index=False)
    shape_export_df.to_excel(writer, sheet_name='Potential_Route_Id_Matches',index=False)